In [171]:
## 데이터 분석 관련
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

## 데이터 시각화 관련
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') # matplotlib의 스타일에 관련한 함
## 그래프 출력에 필요한 IPython 명령어
%matplotlib inline 

## Scikit-Learn의 다양한 머신러닝 모듈을 불러옵니다.
## 분류 알고리즘 중에서 선형회귀, 서포트벡터머신, 랜덤포레스트, K-최근접이웃 알고리즘을 사용해보려고 합니다.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

train_df = pd.read_csv("C:/Users/The9/Documents/My_Study/data/titanic-train.csv")
test_df = pd.read_csv("C:/Users/The9/Documents/My_Study/data/titanic-test.csv")
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [172]:
# 위의 정보로 볼때 번호는 큰 의미를 가지지 않고, 이름과 티켓의 경우에는 불규칙성이 많아 처리하기 어려울 것 같습니다.
# 데이터의 정보는 info 메서드로 확인할 수 있습니다. 훈련 데이터와 테스트 데이터를 확인
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [173]:
test_df.info()
print('*'*50)
print(test_df['Fare'].isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
**************************************************
1


In [174]:
print(test_df.isna().sum()/len(test_df)) # 객실번호 78%가 nan

PassengerId    0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.205742
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.002392
Cabin          0.782297
Embarked       0.000000
dtype: float64


In [175]:
# 위 결과에서 각각의 데이터 개수는 891개. 418개인 것을 확인할 수 있습니다. 특성은 각각 12개 11개입니다.
# 그 이유는 훈련 데이터는 생존 여부를 알고 있기 때문입니다.
# 여기서 주의깊게 봐야할 부분은 다음과 같습니다.
# 
# 각 데이터는 빈 부분이 있는가?
# 빈 부분이 있다면, drop할 것인가 아니면 default값으로 채워넣을 것인가
# cabin, Age, Embarked 세 항목에 주의
# 데이터는 float64로 변환할 수 있는가
# 아니라면 범주형 데이터로 만들 수 있는가
# 
# 필요없는 부분이라고 생각되는 부분을 지웁니다.
# 여기서는 PassengerID와 이름, 티켓을 지웁니다. 이름과 티켓에서 가져올 수 있는 데이터는 없기 때문입니다.
# 하지만 이 문제에서 결과물은 'PassengerId', 'Survived' 요소가 필요하므로 훈련데이터에서만 삭제합니다.
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
test_df = test_df.drop(['Name','Ticket'], axis=1)

In [176]:
print(train_df.head(2))
print('-'*30)
print(test_df.head(2))

   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0         0       3    male  22.0      1      0   7.2500   NaN        S
1         1       1  female  38.0      1      0  71.2833   C85        C
------------------------------
   PassengerId  Pclass     Sex   Age  SibSp  Parch    Fare Cabin Embarked
0          892       3    male  34.5      0      0  7.8292   NaN        Q
1          893       3  female  47.0      1      0  7.0000   NaN        S


In [177]:
# 객실등급 별 몇명이 있나 ?
train_df['Pclass'].value_counts() # 3 :491, 1: 216, 2: 184

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

In [178]:
# 3. 데이터 하나하나 처리하기
# 이제 남은 데이터 종류는 다음과 같습니다.
# Pclass
# Sex
# Age
# SibSp
# Parch
# Fare
# Cabin
# Embarked
# 이제 순서대로 보도록 하겠습니다.

In [179]:
#  1, 2, 3 등급은 경우에 따라 다를 수 있지만 연속적인 정보가 아니며, 각 차이 또한 균등하지 않습니다.
# 그렇기에 범주형(카테고리) 데이터로 인식하고 인코딩해야합니다.
# 범주형 데이터이므로 one-hot-encoding을 pd.get_dummies() 메서드로 인코딩
# get_dummies는 해당 데이터를 3열로 만들어서 이항(Boolean) 으로 명시됨
pclass_train_dummies = pd.get_dummies(train_df['Pclass']) 
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])
pclass_train_dummies
pclass_test_dummies


,1,2,3
0,False,False,True
1,False,False,True
2,False,True,False
3,False,False,True
4,False,False,True
...,...,...,...
413,False,False,True
414,True,False,False
415,False,False,True
416,False,False,True


In [180]:
train_df.drop('Pclass', axis=1, inplace=True) # 원본의 Pclass를 없앰
test_df.drop('Pclass', axis=1, inplace=True)
print(train_df.head(3))
print('-'*75)
print(test_df.head(3))

   Survived     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0         0    male  22.0      1      0   7.2500   NaN        S
1         1  female  38.0      1      0  71.2833   C85        C
2         1  female  26.0      0      0   7.9250   NaN        S
---------------------------------------------------------------------------
   PassengerId     Sex   Age  SibSp  Parch    Fare Cabin Embarked
0          892    male  34.5      0      0  7.8292   NaN        Q
1          893  female  47.0      1      0  7.0000   NaN        S
2          894    male  62.0      0      0  9.6875   NaN        Q


In [181]:
train_df = train_df.join(pclass_train_dummies) # 범주형 데이터 더미 이어붙이기
test_df = test_df.join(pclass_test_dummies)
print(train_df.head(3)) # 범주형으로 개별로 데이터가 변환 됨
print('-'*75)
print(test_df.head(3))

   Survived     Sex   Age  SibSp  Parch     Fare Cabin Embarked      1      2   
0         0    male  22.0      1      0   7.2500   NaN        S  False  False  \
1         1  female  38.0      1      0  71.2833   C85        C   True  False   
2         1  female  26.0      0      0   7.9250   NaN        S  False  False   

       3  
0   True  
1  False  
2   True  
---------------------------------------------------------------------------
   PassengerId     Sex   Age  SibSp  Parch    Fare Cabin Embarked      1   
0          892    male  34.5      0      0  7.8292   NaN        Q  False  \
1          893  female  47.0      1      0  7.0000   NaN        S  False   
2          894    male  62.0      0      0  9.6875   NaN        Q  False   

       2      3  
0  False   True  
1  False   True  
2   True  False  


In [182]:
# 1,2,3 컬럼을 각 등급 객실로 바꿔주기
train_df.rename(columns={1: '1등 객실', 2: '2등 객실', 3:'3등 객실'}, inplace=True)
test_df.rename(columns={1: '1등 객실', 2: '2등 객실', 3:'3등 객실'}, inplace=True)
print(train_df.head(2))
print('-'*75)
print(test_df.head(2))

   Survived     Sex   Age  SibSp  Parch     Fare Cabin Embarked  1등 객실  2등 객실   
0         0    male  22.0      1      0   7.2500   NaN        S  False  False  \
1         1  female  38.0      1      0  71.2833   C85        C   True  False   

   3등 객실  
0   True  
1  False  
---------------------------------------------------------------------------
   PassengerId     Sex   Age  SibSp  Parch    Fare Cabin Embarked  1등 객실   
0          892    male  34.5      0      0  7.8292   NaN        Q  False  \
1          893  female  47.0      1      0  7.0000   NaN        S  False   

   2등 객실  3등 객실  
0  False   True  
1  False   True  


In [183]:
# Sex는 성별입니다. 남과 여로 나뉘므로 이 또한 one-hot-encoding을 진행해봅시다.
# 더미 변수에 Sex에 대한 이항을 넣었음
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])
print(sex_train_dummies.head(2))
print('-'*75)
print(sex_test_dummies.head(2))

   female   male
0   False   True
1    True  False
---------------------------------------------------------------------------
   female   male
0   False   True
1    True  False


In [184]:
# Sex를 없애기 위해 더미 변수에 원본에 넣을 'Female', 'Male' 컬럼을 만들어줌
sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']
print(sex_train_dummies.columns)
print(sex_test_dummies.columns)

Index(['Female', 'Male'], dtype='object')
Index(['Female', 'Male'], dtype='object')


In [185]:
# Sex 컬럼 삭제
train_df.drop(['Sex'], axis=1, inplace=True)
test_df.drop(['Sex'], axis=1, inplace=True)
print(train_df)
print(test_df)

     Survived   Age  SibSp  Parch     Fare Cabin Embarked  1등 객실  2등 객실  3등 객실
0           0  22.0      1      0   7.2500   NaN        S  False  False   True
1           1  38.0      1      0  71.2833   C85        C   True  False  False
2           1  26.0      0      0   7.9250   NaN        S  False  False   True
3           1  35.0      1      0  53.1000  C123        S   True  False  False
4           0  35.0      0      0   8.0500   NaN        S  False  False   True
..        ...   ...    ...    ...      ...   ...      ...    ...    ...    ...
886         0  27.0      0      0  13.0000   NaN        S  False   True  False
887         1  19.0      0      0  30.0000   B42        S   True  False  False
888         0   NaN      1      2  23.4500   NaN        S  False  False   True
889         1  26.0      0      0  30.0000  C148        C   True  False  False
890         0  32.0      0      0   7.7500   NaN        Q  False  False   True

[891 rows x 10 columns]
     PassengerId   Age  Sib

In [186]:
# 원본에 없앤 Sex를 대신할 Fmale, Male컬럼을 넣어줌
train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)
print(train_df.head(2))
print('-'*75)
print(test_df.head(2))

   Survived   Age  SibSp  Parch     Fare Cabin Embarked  1등 객실  2등 객실  3등 객실   
0         0  22.0      1      0   7.2500   NaN        S  False  False   True  \
1         1  38.0      1      0  71.2833   C85        C   True  False  False   

   Female   Male  
0   False   True  
1    True  False  
---------------------------------------------------------------------------
   PassengerId   Age  SibSp  Parch    Fare Cabin Embarked  1등 객실  2등 객실   
0          892  34.5      0      0  7.8292   NaN        Q  False  False  \
1          893  47.0      1      0  7.0000   NaN        S  False  False   

   3등 객실  Female   Male  
0   True   False   True  
1   True    True  False  


In [187]:
# 3.3 Age
# 나이는 연속형 데이터이므로, 큰 처리가 필요없습니다.  
# (카테고리화를 하여 일부 알고리즘에 더 유용한 결과를 만들 수 있습니다.)  
# 하지만 일부 NaN 데이터가 있으니 이를 채울 수 있는 방법에 대해서 생각해봅시다.

# 랜덤  
# 평균값  
# 중간값  
# 데이터 버리기  
# 저는 일단은 평균값으로 채우도록 하겠습니다.  
# 데이터의 통일성을 가지기 위해 train 데이터셋의 평균값으로 훈련, 테스트 데이터셋을 채우겠습니다.

In [188]:
train_df["Age"].fillna(train_df["Age"].mean() , inplace=True)
test_df["Age"].fillna(train_df["Age"].mean() , inplace=True)
print(train_df.head(2))
print('-'*75)
print(test_df.head(2))

   Survived   Age  SibSp  Parch     Fare Cabin Embarked  1등 객실  2등 객실  3등 객실   
0         0  22.0      1      0   7.2500   NaN        S  False  False   True  \
1         1  38.0      1      0  71.2833   C85        C   True  False  False   

   Female   Male  
0   False   True  
1    True  False  
---------------------------------------------------------------------------
   PassengerId   Age  SibSp  Parch    Fare Cabin Embarked  1등 객실  2등 객실   
0          892  34.5      0      0  7.8292   NaN        Q  False  False  \
1          893  47.0      1      0  7.0000   NaN        S  False  False   

   3등 객실  Female   Male  
0   True   False   True  
1   True    True  False  


In [189]:
# 3.4 SibSp & Panch
# 형제 자매와 부모님은 가족으로 함께 처리할 수 있습니다. 하지만 마찬가지로 바꿀 필요는 없습니다.

# 3.5 Fare
# Fare은 탑승료입니다. 신기하게 test 데이터셋에 1개의 데이터가 비어있습니다. 아마 디카프리오인듯 합니다. :-) 
test_df[test_df['Fare'].isnull()]

,PassengerId,Age,SibSp,Parch,Fare,Cabin,Embarked,1등 객실,2등 객실,3등 객실,Female,Male
152,1044,60.5,0,0,NaN,NaN,S,False,False,True,False,True


In [190]:
# 빈 부분을 fillna 메서드로 채우겠습니다. 누락이 아닌 무단 탑승이라 생각하고 0으로 입력
test_df["Fare"].fillna(0, inplace=True)
test_df[test_df['Fare']==0] # 다른 2명은 1등 객실이라 요금 0원 혜택이 있거나 선원 가족?

,PassengerId,Age,SibSp,Parch,Fare,Cabin,Embarked,1등 객실,2등 객실,3등 객실,Female,Male
152,1044,60.500000,0,0,0.0,NaN,S,False,False,True,False,True
266,1158,29.699118,0,0,0.0,NaN,S,True,False,False,False,True
372,1264,49.000000,0,0,0.0,B52 B54 B56,S,True,False,False,False,True


In [191]:
# 3.6 Cabin
# Cabin은 객실입니다. NaN이 대부분인 데이터이므로 버립시다.
# 이 데이터를 살리는 것은 너무 어려운 일입니다.
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)
print(train_df.head(2))
print('-'*75)
print(test_df.head(2))

   Survived   Age  SibSp  Parch     Fare Embarked  1등 객실  2등 객실  3등 객실   
0         0  22.0      1      0   7.2500        S  False  False   True  \
1         1  38.0      1      0  71.2833        C   True  False  False   

   Female   Male  
0   False   True  
1    True  False  
---------------------------------------------------------------------------
   PassengerId   Age  SibSp  Parch    Fare Embarked  1등 객실  2등 객실  3등 객실   
0          892  34.5      0      0  7.8292        Q  False  False   True  \
1          893  47.0      1      0  7.0000        S  False  False   True   

   Female   Male  
0   False   True  
1    True  False  


In [192]:
# 3.7 Embarked
# Embarked는 탑승 항구를 의미합니다. 우선 데이터를 확인해보겠습니다.
print(train_df['Embarked'].value_counts())
print('-'*25)
print(test_df['Embarked'].value_counts())

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64
-------------------------
Embarked
S    270
C    102
Q     46
Name: count, dtype: int64


In [193]:
# S가 대다수이고 일부 데이터가 비어있는 것을 알 수 있습니다. 빈 부분은 S로 우선 채우고 시작합시다
train_df["Embarked"].fillna('S', inplace=True)
test_df["Embarked"].fillna('S', inplace=True)
print(train_df['Embarked'].info())
print('-'*50)
print(test_df['Embarked'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: Embarked
Non-Null Count  Dtype 
--------------  ----- 
891 non-null    object
dtypes: object(1)
memory usage: 7.1+ KB
None
--------------------------------------------------
<class 'pandas.core.series.Series'>
RangeIndex: 418 entries, 0 to 417
Series name: Embarked
Non-Null Count  Dtype 
--------------  ----- 
418 non-null    object
dtypes: object(1)
memory usage: 3.4+ KB
None


In [194]:
# 아까 했던 탑승항구 S,Q,C 별로 칼럼을 만들어주자 
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop('Embarked', axis=1, inplace=True)
test_df.drop('Embarked', axis=1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)
print(train_df.head(2))
print('-'*50)
print(test_df.head(2))

   Survived   Age  SibSp  Parch     Fare  1등 객실  2등 객실  3등 객실  Female   Male   
0         0  22.0      1      0   7.2500  False  False   True   False   True  \
1         1  38.0      1      0  71.2833   True  False  False    True  False   

       S      C      Q  
0  False  False   True  
1   True  False  False  
--------------------------------------------------
   PassengerId   Age  SibSp  Parch    Fare  1등 객실  2등 객실  3등 객실  Female   
0          892  34.5      0      0  7.8292  False  False   True   False  \
1          893  47.0      1      0  7.0000  False  False   True    True   

    Male      S      C      Q  
0   True  False   True  False  
1  False  False  False   True  


In [196]:
# 4. 데이터 나누기
# 이제 학습용 데이터를 위해 데이터를 나누어야합니다.
# (정보, 생존 여부)와 같은 형태를 위하여 다음과 같이 데이터를 나눕니다.
X_train = train_df.drop("Survived",axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId",axis=1).copy()

In [197]:
# 5. 머신러닝 알고리즘 적용하기
# 이제 로지스틱 회귀, SVC, 랜덤 포레스트, K-최근접 이웃 알고리즘을 각각 적용해봅시다.

# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
logreg.score(X_train, Y_train)

c:\Users\The9\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8058361391694725

In [202]:
# Support Vector Machines
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
svc.score(X_train, Y_train)

0.6868686868686869

In [203]:
# Random Forests
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)

0.9820426487093153

In [204]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
knn.score(X_train, Y_train)

0.835016835016835

In [ ]:
# 6. 제출용 파일 만들기
# 랜덤 포레스트가 가장 좋은 결과를 내는 것을 알 수 있습니다. 그 결과로 submission 파일을 만들어 제출해봅시다
# Random Forests

# random_forest = RandomForestClassifier(n_estimators=100)
# random_forest.fit(X_train, Y_train)
# Y_pred = random_forest.predict(X_test)
# random_forest.score(X_train, Y_train)

# submission = pd.DataFrame({
#         "PassengerId": test_df["PassengerId"],
#         "Survived": Y_pred
#     })
# submission.to_csv('titanic.csv', index=False)